### *Loading the fine-tuned model*

### *Installing the requirements*

In [ ]:
! pip install -qqq datasets rouge-score bert-score

In [ ]:
import pandas as pd
import torch
import bert_score
from datasets import Dataset
from rouge_score import rouge_scorer

### *Loading the dataset*

In [ ]:
#@title Function to calculate ROUGE scores

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def calculate_rouge_scores(reference, hypothesis):
    scores = scorer.score(reference, hypothesis)
    return {
        'rouge1_precision': scores['rouge1'].precision,
        'rouge1_recall': scores['rouge1'].recall,
        'rouge1_f1': scores['rouge1'].fmeasure,
        'rouge2_precision': scores['rouge2'].precision,
        'rouge2_recall': scores['rouge2'].recall,
        'rouge2_f1': scores['rouge2'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall,
        'rougeL_f1': scores['rougeL'].fmeasure
    }

In [ ]:
test_df = pd.read_csv("Model_output.csv")
test_df_dataset = Dataset.from_pandas(test_df)

In [ ]:
from tqdm import tqdm

predictions = df["model_predictions"].tolist()

# Calculate ROUGE scores
results = []

for i, (row, prediction) in tqdm(enumerate(zip(test_df_dataset, predictions)), total=len(predictions), desc="Calculating ROUGE scores"):
    summary = row["Perspective-based Summary"]
    rouge_scores = calculate_rouge_scores(summary, prediction)
    results.append(rouge_scores)

In [ ]:
# Convert the results into a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
# Merge the actual dataset and the adjusted scores dataset on the index
merged_df = pd.merge(test_df, results_df, left_index=True, right_index=True)

### *BERT Score*

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
summary = merged_df["Perspective-based Summary"].tolist() # actual summary

# Compute BERTScore
precision, recall, f1 = bert_score.score(summary, predictions, lang='en', verbose=True, device=device)

In [ ]:
# Add BERTScore to the DataFrame
merged_df['BERTScore_Precision'] = precision.numpy()
merged_df['BERTScore_Recall'] = recall.numpy()
merged_df['BERTScore_F1'] = f1.numpy()

### *Saving the scores*

In [ ]:
# Save the result to a new CSV file
merged_df.to_csv('Evaluation_scores.csv', index=False)

### *Overall Average Scores*

In [ ]:
df = pd.read_csv('Evaluation_scores.csv')

In [ ]:
# Calculate aggregated results
aggregated_results = {
    'rouge1_precision': df['rouge1_precision'].mean(),
    'rouge1_recall': df['rouge1_recall'].mean(),
    'rouge1_f1': df['rouge1_f1'].mean(),
    'rouge2_precision': df['rouge2_precision'].mean(),
    'rouge2_recall': df['rouge2_recall'].mean(),
    'rouge2_f1': df['rouge2_f1'].mean(),
    'rougeL_precision': df['rougeL_precision'].mean(),
    'rougeL_recall': df['rougeL_recall'].mean(),
    'rougeL_f1': df['rougeL_f1'].mean(),
    'BERTScore_Precision': df['BERTScore_Precision'].mean(),
    'BERTScore_Recall': df['BERTScore_Recall'].mean(),
    'BERTScore_F1': df['BERTScore_F1'].mean()
}
aggregated_rouge_scores = pd.DataFrame(aggregated_results, index=['Value'])
aggregated_rouge_scores